In [ ]:
#%matplotlib inline
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**9
run_time = 4*t_max
input_dims = (1,32,32)
weight_dims = (1,6,1,5,5)
weight_e = 101
weight_acc = 2**7
model_args = {'weight_e':weight_e, 'weight_acc':weight_acc}

np.random.seed(seed=47)
weights1 = (np.random.rand(np.product(dims2),np.product(dims1)) - 0.5) / 4
weights2 = (np.random.rand(n_outputs,np.product(dims2)) - 0.5) / 4
biases1 = (np.random.rand(np.product(dims2)) - 0.5) / 2
biases2 = (np.random.rand(n_outputs) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims1, **model_args),
    layers.Dense(weights=weights1, biases=biases1, **model_args),
    layers.Dense(weights=weights2, biases=biases2, **model_args),
    layers.MonitorLayer(**model_args),
])

values = np.random.rand(np.product(dims1)) # np.ones((np.product(dims))) * 0.5
inputs = quartz.decode_values_into_spike_input(values, t_max)

# quantized_values = (values*t_max).round()/t_max
# quantized_weights = (weight_acc_real*weights).round()/weight_acc_real
# quantized_biases = (biases*t_max).round()/t_max

# pt_model = nn.Sequential(
#     nn.Linear(in_features=np.product(dims1), out_features=np.product(dims2)), 
#     nn.ReLU(),
#     nn.Linear(in_features=np.product(dims2), out_features=n_outputs), 
#     nn.ReLU(),
# )
# pt_model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
# pt_model[0].bias = torch.nn.Parameter(torch.tensor((quantized_biases)))
#model_output = pt_model(torch.tensor(quantized_values)).detach().numpy()
loihi_model

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
loihi_model.core_ids


In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = ( 1,10,10)
weight_dims = ( 6,1,3,3)
weight_e = 255

# (( 1,32,32), ( 6,1,5,5), 251),
# (( 6,14,14), (8,6,5,5), 251),
t_max = 2**9
weight_acc = 128
model_args = {'weight_e':weight_e, 'weight_acc':weight_acc}
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=47)
weights = (np.random.rand(*weight_dims)-0.5) / 4
biases = (np.random.rand(weight_dims[0])-0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims, **model_args),
    layers.Conv2D(weights=weights, biases=biases, **model_args),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride, **model_args),
    layers.MonitorLayer(**model_args),
])

pool0 = quartz.probe(loihi_model.layers[2].blocks[0])

values = np.random.rand(np.product(input_dims))
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
#model_output.flatten()

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
pool0.plot()

In [ ]:
pool0.output()